<a href="https://colab.research.google.com/github/nullvoid-ky/introduction-to-machine-learning-and-deep-learning/blob/main/DataClean%26Augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===== Setup & Installs (Kaggle usually has most of these; safe to re-run) =====
!pip -q install kagglehub shap lightgbm xgboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance

import shap
import warnings
warnings.filterwarnings('ignore')


In [2]:
# ถ้าในสภาพแวดล้อมคุณยังไม่มี ให้รันก่อน (Kaggle มักมีอยู่แล้ว)
!pip install lightgbm xgboost -q


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("utkarshx27/american-companies-bankruptcy-prediction-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'american-companies-bankruptcy-prediction-dataset' dataset.
Path to dataset files: /kaggle/input/american-companies-bankruptcy-prediction-dataset


In [4]:
from kagglehub import KaggleDatasetAdapter, load_dataset
import pandas as pd

# Load the dataset
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Set the CSV file path **inside** the dataset (adjust if needed)
# Explore the dataset directory printed below to confirm the file name.
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
file_path = "/kaggle/input/american-companies-bankruptcy-prediction-dataset/american_bankruptcy.csv"

df = pd.read_csv(file_path)

print("Loaded shape:", df.shape)
print("Columns:\n", list(df.columns))
df.head()

Loaded shape: (78682, 21)
Columns:
 ['company_name', 'status_label', 'year', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18']


,company_name,status_label,year,X1,X2,X3,X4,X5,X6,X7,...,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,C_1,alive,1999,511.267,833.107,18.373,89.031,336.018,35.163,128.348,...,1024.333,740.998,180.447,70.658,191.226,163.816,201.026,1024.333,401.483,935.302
1,C_1,alive,2000,485.856,713.811,18.577,64.367,320.590,18.531,115.187,...,874.255,701.854,179.987,45.790,160.444,125.392,204.065,874.255,361.642,809.888
2,C_1,alive,2001,436.656,526.477,22.496,27.207,286.588,-58.939,77.528,...,638.721,710.199,217.699,4.711,112.244,150.464,139.603,638.721,399.964,611.514
3,C_1,alive,2002,396.412,496.747,27.172,30.745,259.954,-12.410,66.322,...,606.337,686.621,164.658,3.573,109.590,203.575,124.106,606.337,391.633,575.592
4,C_1,alive,2003,432.204,523.302,26.680,47.491,247.245,3.504,104.661,...,651.958,709.292,248.666,20.811,128.656,131.261,131.884,651.958,407.608,604.467


In [5]:
import numpy as np
import pandas as pd

FEATURES = ["X8","X17","X3","X11","X10","X1","X6"]
TARGET   = "status_label"
COMPANY  = "company_name"   # ถ้าไม่มีคอลัมน์นี้ โค้ดจะ fallback อัตโนมัติ

# ---------- helpers ----------
def _check_cols(df, features, target, company_col=None):
    req = set(features+[target])
    miss = list(req - set(df.columns))
    if miss:
        raise ValueError(f"Missing columns: {miss}")
    if company_col is not None and company_col not in df.columns:
        company_col = None
    return company_col

def _clip_like(df_aug, df_ref, features):
    # คลิปค่าให้อยู่ในช่วง min/max ของทั้ง dataset เพื่อไม่ให้ synthetic หลุดโลก
    mins = df_ref[features].min()
    maxs = df_ref[features].max()
    df_aug[features] = df_aug[features].clip(lower=mins, upper=maxs, axis=1)
    return df_aug

def augment_failed(df, features=FEATURES, target=TARGET, company_col=COMPANY,
                   n_new=0, random_state=42):
    """สร้างแถว failed (target=1) เพิ่ม n_new แถว ด้วย 3 วิธี: uniform jitter, gaussian noise, mixup."""
    if n_new <= 0:
        return pd.DataFrame(columns=df.columns)

    rng = np.random.default_rng(random_state)
    df1 = df[df[target]==1].copy()
    if df1.empty:
        raise ValueError("No failed rows to augment.")

    # เตรียมสัดส่วน 3 วิธี แบ่งพอๆ กัน
    n1 = int(np.floor(n_new/3))
    n2 = int(np.floor(n_new/3))
    n3 = n_new - n1 - n2

    # 1) Uniform jitter ±0.5%
    take1 = df1.sample(n=max(n1,0), replace=True, random_state=random_state).copy()
    if not take1.empty:
        jitter = rng.uniform(-0.005, 0.005, size=(len(take1), len(features)))
        take1.loc[:, features] = take1[features].values * (1.0 + jitter)
        take1["aug_method"] = "uniform_0p5pct"

    # 2) Gaussian noise (σ = 0.5% ของ std ต่อคอลัมน์คลาส failed)
    take2 = df1.sample(n=max(n2,0), replace=True, random_state=random_state+1).copy()
    if not take2.empty:
        stds = df1[features].std().replace(0, 1e-12)
        noise = rng.normal(loc=0.0, scale=(0.005*stds.values), size=(len(take2), len(features)))
        take2.loc[:, features] = take2[features].values + noise
        take2["aug_method"] = "gaussian_0p5pct_std"

    # 3) Mixup (λ ~ Beta(0.4, 0.4))
    take3 = pd.DataFrame(columns=df.columns)
    if n3 > 0:
        a = df1.sample(n=n3, replace=True, random_state=random_state+2)
        b = df1.sample(n=n3, replace=True, random_state=random_state+3)
        lam = rng.beta(0.4, 0.4, size=n3).reshape(-1,1)
        mix_vals = lam * a[features].values + (1-lam) * b[features].values
        take3 = a.copy()
        take3.loc[:, features] = mix_vals
        take3["aug_method"] = "mixup_beta_0.4"

    df_aug = pd.concat([x for x in [take1, take2, take3] if not x.empty], ignore_index=True)
    df_aug[target] = 1

    # ทำให้ค่ามีเหตุผล (clip ตาม min/max ทั้ง df)
    df_aug = _clip_like(df_aug, df, features)

    # ถ้าอยากตั้งชื่อบริษัทให้รู้ว่าเป็น aug (เฉพาะมี company_name)
    if company_col is not None:
        df_aug[company_col] = df_aug[company_col].astype(str)  # คงบริษัทเดิมไว้ (สมจริงกว่า)
        # หรือจะเติม suffix:
        # df_aug[company_col] = df_aug[company_col].astype(str) + "_aug"

    return df_aug

def downsample_alive_diverse(df, keep_n, features=FEATURES, target=TARGET, company_col=COMPANY, random_state=42):
    """คัด 'alive'=0 ให้เหลือ keep_n โดยกระจาย 'บริษัท' ให้หลากหลายที่สุดเท่าที่ทำได้"""
    df0 = df[df[target]==0].copy()
    if company_col is None:
        # ไม่มี company_name → สุ่มตรงๆ
        return df0.sample(n=min(keep_n, len(df0)), replace=False, random_state=rs)

    rs = np.random.RandomState(random_state)

    # 1) เลือกอย่างน้อย 1 แถวต่่อบริษัท (ครอบคลุมบริษัทให้เยอะสุด)
    pick1 = df0.groupby(company_col, group_keys=False).apply(lambda g: g.sample(1, random_state=rs)).reset_index(drop=True)
    if len(pick1) >= keep_n:
        # บริษัทเยอะกว่าโควต้า → สุ่มเลือกบริษัทบางส่วน
        # เลือกบริษัทสุ่ม keep_n แห่ง แล้วหยิบ 1 แถวจากแต่ละบริษัท
        chosen_companies = rs.choice(pick1[company_col].unique(), size=keep_n, replace=False)
        kept = pick1[pick1[company_col].isin(chosen_companies)].copy()
        return kept.reset_index(drop=True)

    # 2) ถ้ายังไม่พอ ต้องเติมจากแถวนอกเหนือจากที่เลือกไปแล้ว
    rem_need = keep_n - len(pick1)
    remaining = df0.drop(index=pick1.index)
    if rem_need > len(remaining):
        rem_need = len(remaining)
    pick2 = remaining.sample(n=rem_need, replace=False, random_state=rs)
    kept = pd.concat([pick1, pick2], ignore_index=True)
    return kept.sample(frac=1.0, random_state=rs).reset_index(drop=True)

def balance_40_50(df, features=FEATURES, target=TARGET, company_col=COMPANY,
                  target_ratio=0.45, min_ratio=0.40, random_state=42):
    """
    ทำให้สัดส่วน failed อยู่ในช่วง ~40–50%.
    กลยุทธ์:
      1) คำนวณเพดานจำนวน alive ที่ 'ควรเก็บ' เพื่อให้ >= min_ratio โดยไม่ต้อง augment: N_new <= P*(1-min)/min
      2) downsample alive โดยกระจายบริษัท
      3) augment failed เพิ่มให้ถึง target_ratio ด้วย 3 วิธี noise
    """
    company_col = _check_cols(df, features, target, company_col)

    df = df.copy()
    df[target] = df[target].astype(int)

    P = int((df[target]==1).sum())
    N = int((df[target]==0).sum())

    if P == 0:
        raise ValueError("No failed rows in the dataset.")
    if N == 0:
        return df  # already all failed

    # 1) หาโควต้าจำนวน alive สูงสุดที่ยังทำให้ ratio >= min_ratio ถ้าไม่ augment
    #    เงื่อนไข: P / (P + N_keep) >= min_ratio  ->  N_keep <= P*(1-min)/min
    N_keep_cap = int(np.floor(P*(1 - min_ratio) / min_ratio))
    N_keep_cap = max(1, min(N_keep_cap, N))  # ป้องกันขอบ
    # เลือกจำนวน alive ที่เราจะ "เก็บ" จริง ๆ (อาจเท่ากับ cap)
    N_keep = N_keep_cap

    # 2) คัด alive ให้เหลือ N_keep โดยกระจายบริษัท
    kept_alive = downsample_alive_diverse(df, keep_n=N_keep, features=features, target=target,
                                          company_col=company_col, random_state=random_state)

    # 3) จำนวน failed ที่ "ต้องมี" เพื่อได้ target_ratio:
    #    r = (P + A) / (P + A + N_keep) = target_ratio  ->  A = r*N_keep/(1-r) - P
    A_need = int(np.ceil(target_ratio * N_keep / (1 - target_ratio) - P))
    A_need = max(0, A_need)

    df_failed = df[df[target]==1]
    df_aug = augment_failed(
        pd.concat([df_failed, kept_alive], ignore_index=True),  # ให้ clip ช่วงตามทั้งชุดใหม่
        features=features, target=target, company_col=company_col,
        n_new=A_need, random_state=random_state
    )

    # รวมชุดใหม่: failed เดิม + failed augment + alive ที่คัดไว้
    new_df = pd.concat([df_failed, df_aug, kept_alive], ignore_index=True)
    new_df = new_df.sample(frac=1.0, random_state=random_state).reset_index(drop=True)

    # รายงานผล
    P2 = int((new_df[target]==1).sum()); N2 = int((new_df[target]==0).sum())
    ratio = P2 / (P2 + N2)
    print(f"Before: failed={P}, alive={N}, ratio={P/(P+N):.3f}")
    print(f"Keep alive = {N_keep}  |  Aug failed = {A_need}")
    print(f"After:  failed={P2}, alive={N2}, ratio={ratio:.3f}  (~{ratio*100:.1f}%)")
    if company_col is not None:
        uniq_alive = new_df.loc[new_df[target]==0, company_col].nunique()
        print(f"Unique companies among KEPT alive: {uniq_alive}")
    return new_df


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TARGET = "status_label"
FEATURES = ["X8","X17","X3","X11","X10","X1","X6"]

# 0) ตรวจว่าคอลัมน์ครบไหม
missing = [c for c in FEATURES+[TARGET] if c not in df.columns]
if missing:
    raise ValueError(f"❌ Missing columns: {missing}")

# 1) ฟังก์ชัน normalize label ให้เป็น 0/1 แบบทนทาน
def normalize_status(x):
    if pd.isna(x):
        return np.nan
    t = str(x).strip().lower()
    # ตัวเลขที่มาเป็นสตริง หรือ float 0.0/1.0
    if t in {"0","1"}:
        return int(t)
    try:
        # กรณีเป็น 0.0/1.0 จริง ๆ
        f = float(t)
        if f in (0.0, 1.0):
            return int(f)
    except:
        pass
    # แม็พคำยอดฮิต
    direct = {
        "alive": 0, "non-bankrupt": 0, "nonbankrupt": 0, "healthy": 0, "normal": 0,
        "failed": 1, "fail": 1, "bankrupt": 1, "bankruptcy": 1, "went_bankrupt": 1,
        "yes": 1, "y": 1, "true": 1,
        "no": 0, "n": 0, "false": 0
    }
    if t in direct:
        return direct[t]
    # สุดท้าย ถ้าระบุไม่ถูก ให้คืน NaN เพื่อตรวจสอบ
    return np.nan

y_norm = df[TARGET].apply(normalize_status)

# 2) เช็คค่าที่แปลงไม่ได้ (จะเป็น NaN)
bad_mask = y_norm.isna()
if bad_mask.any():
    print("⚠️ พบ label ที่ไม่รู้จัก (ตัวอย่าง top 20):")
    print(df.loc[bad_mask, TARGET].value_counts().head(20))
    # ทางเลือก: ตัดแถวที่ label ไม่ชัดเจนทิ้งไปก่อน
    df = df.loc[~bad_mask].copy()
    y_norm = y_norm.loc[~bad_mask]

# 3) เขียนกลับเป็นตัวเลข 0/1
df[TARGET] = y_norm.astype(int)


balanced_train = balance_40_50(
    df=train_df,
    features=["X8","X17","X3","X11","X10","X1","X6"],
    target="status_label",
    company_col="company_name",   # ถ้าไม่มีคอลัมน์นี้ ปล่อยไว้หรือส่ง None
    target_ratio=0.5,            # ปรับเป็น 0.50 หากอยาก 50/50
    min_ratio=0.40,
    random_state=42
)

Before: failed=4176, alive=58769, ratio=0.066
Keep alive = 6263  |  Aug failed = 2087
After:  failed=6263, alive=6263, ratio=0.500  (~50.0%)
Unique companies among KEPT alive: 6263


In [18]:

# 4) แยก train/test แบบ stratified
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df[TARGET]
)

print("✅ label cleaned & split done")
print("Train ratio:", train_df[TARGET].mean().round(3),
      "| Test ratio:", test_df[TARGET].mean().round(3))

✅ label cleaned & split done
Train ratio: 0.066 | Test ratio: 0.066


In [15]:
balanced_train

,company_name,status_label,year,X1,X2,X3,X4,X5,X6,X7,...,X10,X11,X12,X13,X14,X15,X16,X17,X18,aug_method
0,C_792,0,2018,481.867000,1026.906,37.855000,100.023,56.437,2.928000,301.330,...,992.417000,282.003000,62.168,306.662,219.650,339.600,1333.568,522.230000,1233.545,NaN
1,C_2689,0,2015,0.547000,0.485,0.005000,-0.437,0.192,-6.688000,0.096,...,0.658000,0.000000,-0.442,0.049,1.906,-1.507,0.534,1.906000,0.971,NaN
2,C_8208,0,2012,1.960000,0.076,0.233000,-5.616,0.000,-6.039000,0.102,...,6.359000,2.094000,-5.849,0.425,1.457,-104.807,0.501,3.859000,6.117,NaN
3,C_1122,0,2004,224.750000,379.495,19.520000,43.743,84.044,4.956000,107.908,...,590.114000,186.286000,24.223,53.927,78.745,-99.651,433.422,267.149000,389.679,NaN
4,C_1727,0,2003,5.918000,17.889,0.442000,-1.335,2.920,-2.544000,2.540,...,7.918000,0.506000,-1.777,5.204,8.432,-16.403,23.093,9.371000,24.428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12521,C_8309,0,2015,0.119000,0.220,0.000000,-0.182,0.000,-0.182000,0.000,...,0.119000,0.000000,-0.182,-0.182,0.642,-3.148,0.038,0.642000,0.220,NaN
12522,C_1994,1,2004,295.937617,357.969,13.652994,85.686,8.479,44.212941,60.105,...,1395.767224,300.518349,70.338,85.686,55.174,503.729,443.655,474.302480,357.969,gaussian_0p5pct_std
12523,C_6556,1,2012,62.956945,157.418,42.900472,46.196,0.000,-3.784371,19.035,...,394.956420,128.014038,5.331,116.174,46.977,-1046.405,273.592,203.044764,227.396,gaussian_0p5pct_std
12524,C_7001,1,2005,36.311000,26.210,3.024000,-24.343,0.000,-26.103000,0.000,...,42.981000,7.784000,-27.367,-24.343,6.782,-143.633,1.867,15.970000,26.210,NaN
